In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
!pip install textacy

In [51]:
import spacy
from gensim.models.wrappers import FastText
from gensim.models.keyedvectors import KeyedVectors
nlp = spacy.load('en_core_web_sm')
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import matplotlib.pyplot as plt
import re
import string

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import gensim
import textacy

pd.set_option('display.max_colwidth',100)

In [52]:
df = pd.read_csv('/content/drive/My Drive/MCM_News_Aggregator/uci-news-aggregator.csv',usecols=['TITLE', 'CATEGORY','PUBLISHER'])
df.head(10)

,TITLE,PUBLISHER,CATEGORY
0,"Fed official says weak data caused by weather, should not slow taper",Los Angeles Times,b
1,Fed's Charles Plosser sees high bar for change in pace of tapering,Livemint,b
2,US open: Stocks fall after Fed official hints at accelerated tapering,IFA Magazine,b
3,"Fed risks falling 'behind the curve', Charles Plosser says",IFA Magazine,b
4,Fed's Plosser: Nasty Weather Has Curbed Job Growth,Moneynews,b
5,Plosser: Fed May Have to Accelerate Tapering Pace,NASDAQ,b
6,Fed's Plosser: Taper pace may be too slow,MarketWatch,b
7,Fed's Plosser expects US unemployment to fall to 6.2% by the end of 2014,FXstreet.com,b
8,US jobs growth last month hit by weather:Fed President Charles Plosser,Economic Times,b
9,ECB unlikely to end sterilisation of SMP purchases - traders,Interactive Investor,b


In [53]:
#I do aspire here to have balanced classes
num_of_categories = 45639
shuffled = df.reindex(np.random.permutation(df.index))
e = shuffled[shuffled['CATEGORY'] == 'e'][:num_of_categories]
b = shuffled[shuffled['CATEGORY'] == 'b'][:num_of_categories]
t = shuffled[shuffled['CATEGORY'] == 't'][:num_of_categories]
m = shuffled[shuffled['CATEGORY'] == 'm'][:num_of_categories]
concated = pd.concat([e,b,t,m], ignore_index=True)
#Shuffle the dataset
concated = concated.reindex(np.random.permutation(concated.index))
concated['LABEL'] = 0

In [54]:
concated['title_publish'] = concated.TITLE + " " + concated.PUBLISHER
concated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182556 entries, 15614 to 104364
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   TITLE          182556 non-null  object
 1   PUBLISHER      182556 non-null  object
 2   CATEGORY       182556 non-null  object
 3   LABEL          182556 non-null  int64 
 4   title_publish  182556 non-null  object
dtypes: int64(1), object(4)
memory usage: 8.4+ MB


In [55]:
# Drop null
concated= concated.dropna(subset=['title_publish'])

In [56]:
symbols = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', 
           ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', 
           '#', '*', '+', '\\', '•',  '~', '@', '£', '·', '_', 
           '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', 
           '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', 
           '½', 'à', '…', '“', '★', '”', '–', '●', 'â', '►', 
           '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', 
           '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', 
           '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', 
           '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', 
           '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', 
           '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', 
           '¹', '≤', '‡', '√', ]

def clean_symbol(text):
    text = str(text)
    for symbol in symbols:
        text = text.replace(symbol, '')
    return text

# remove symbols and punctuations 
concated['title_publish'] = concated['title_publish'].apply(lambda x: clean_symbol(x))

In [57]:
# Dictionary of short form words and mispellings
short_forms_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                    "'cause": "because", "could've": "could have", "couldn't": "could not", 
                    "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", 
                    "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                    "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                    "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", 
                    "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                    "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                    "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", 
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                    "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", 
                    "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", 
                    "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", 
                    "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                    "she'll've": "she will have", "she's": "she is", "should've": "should have", 
                    "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", 
                    "that's": "that is", "there'd": "there would", "there'd've": "there would have", 
                    "there's": "there is", "here's": "here is","they'd": "they would", 
                    "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", 
                    "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", 
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", 
                    "we'll've": "we will have", "we're": "we are", "we've": "we have", 
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                    "what're": "what are",  "what's": "what is", "what've": "what have", 
                    "when's": "when is", "when've": "when have", "where'd": "where did", 
                    "where's": "where is", "where've": "where have", "who'll": "who will", 
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", 
                    "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                    "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have",
                    "y'all're": "you all are","y'all've": "you all have","you'd": "you would", 
                    "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", 
                    "you're": "you are", "you've": "you have"}

def clean_shortforms(text):
    clean_text = text
    for shortform in short_forms_dict.keys():
        if re.search(shortform, text):
            clean_text = re.sub(shortform, short_forms_dict[shortform], text)
    return clean_text

# fix short forms
concated['title_publish'] = concated['title_publish'].apply(lambda x: clean_shortforms(x))

In [58]:
concated['title_publish']  = concated['title_publish'] .str.replace('[^\w\s]','')# unpunctuate

In [59]:
parser = English()

BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
def clean_text(text):  
    text = re.sub("[0-9]+", "",text)
    text = re.sub(' +',' ', text)
    text = text.lower() # lowercase text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    tokens = parser(text)
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    tokens=  " ".join([word for word in tokens if (len(word) > 2 and len(word) < 14 ) and word not in string.punctuation and word not in STOP_WORDS])
    return tokens

concated['clean_title_publish'] = concated['title_publish'].apply(lambda x:clean_text(x))

In [60]:
concated.head(5)

,TITLE,PUBLISHER,CATEGORY,LABEL,title_publish,clean_title_publish
15614,Ford's injury not the break 'Star Wars' was looking for,San Francisco Examiner,e,0,Fords injury not the break Star Wars was looking for San Francisco Examiner,fords injury break star wars looking san francisco examiner
153184,"UK scientists slam flu drug, Tamiflu, in ongoing debate",WRTV Indianapolis,m,0,UK scientists slam flu drug Tamiflu in ongoing debate WRTV Indianapolis,scientists slam flu drug tamiflu ongoing debate wrtv indianapolis
105905,Astronomers discover gargantuan rocky planet dubbed 'Mega-Earth',The Detroit News,t,0,Astronomers discover gargantuan rocky planet dubbed MegaEarth The Detroit News,astronomers discover gargantuan rocky planet dubbed megaearth detroit news
31411,Chris Brown Sends 'I Love You' Message to Fans From Jail,AceShowbiz,e,0,Chris Brown Sends I Love You Message to Fans From Jail AceShowbiz,chris brown sends love message fans jail aceshowbiz
11568,Transformers: Age of Extinction retains global box office leadership,PanARMENIAN.Net,e,0,Transformers Age of Extinction retains global box office leadership PanARMENIANNet,transformers age extinction retains global box office leadership


In [61]:
parser = English()

BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
def clean_text(text):  
    text = re.sub("[0-9]+", "",text)
    text = re.sub(' +',' ', text)
    text = text.lower() # lowercase text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    tokens = parser(text)
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    tokens=  [word for word in tokens if (len(word) > 2 and len(word) < 14 ) and word not in string.punctuation and word not in STOP_WORDS]
    return tokens

concated['clean_title_publish1'] = concated['clean_title_publish'].apply(lambda x:clean_text(x))

concated.head(5)

,TITLE,PUBLISHER,CATEGORY,LABEL,title_publish,clean_title_publish,clean_title_publish1
15614,Ford's injury not the break 'Star Wars' was looking for,San Francisco Examiner,e,0,Fords injury not the break Star Wars was looking for San Francisco Examiner,fords injury break star wars looking san francisco examiner,"[fords, injury, break, star, wars, looking, san, francisco, examiner]"
153184,"UK scientists slam flu drug, Tamiflu, in ongoing debate",WRTV Indianapolis,m,0,UK scientists slam flu drug Tamiflu in ongoing debate WRTV Indianapolis,scientists slam flu drug tamiflu ongoing debate wrtv indianapolis,"[scientists, slam, flu, drug, tamiflu, ongoing, debate, wrtv, indianapolis]"
105905,Astronomers discover gargantuan rocky planet dubbed 'Mega-Earth',The Detroit News,t,0,Astronomers discover gargantuan rocky planet dubbed MegaEarth The Detroit News,astronomers discover gargantuan rocky planet dubbed megaearth detroit news,"[astronomers, discover, gargantuan, rocky, planet, dubbed, megaearth, detroit, news]"
31411,Chris Brown Sends 'I Love You' Message to Fans From Jail,AceShowbiz,e,0,Chris Brown Sends I Love You Message to Fans From Jail AceShowbiz,chris brown sends love message fans jail aceshowbiz,"[chris, brown, sends, love, message, fans, jail, aceshowbiz]"
11568,Transformers: Age of Extinction retains global box office leadership,PanARMENIAN.Net,e,0,Transformers Age of Extinction retains global box office leadership PanARMENIANNet,transformers age extinction retains global box office leadership,"[transformers, age, extinction, retains, global, box, office, leadership]"


In [62]:
X_temp=concated['clean_title_publish'].values
y=concated['CATEGORY'].values

In [63]:
def sent_tokenize(sent):
    temp=nlp(sent)
    return [str(token) for token in temp if not token.is_stop]

In [64]:
%%time
X=[sent_tokenize(sent) for sent in X_temp]

CPU times: user 27min 23s, sys: 1.19 s, total: 27min 24s
Wall time: 27min 26s


In [65]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,shuffle=True,stratify=y)

In [66]:
%%time
from gensim.models import FastText

#Dimension of vectors we are generating
EMBEDDING_DIM = 300
#Creating Word Vectors by Word2Vec Method (takes time...)
model = FastText(sentences=concated['clean_title_publish1'], size=EMBEDDING_DIM, window=5, min_count=1)  

CPU times: user 3min 51s, sys: 434 ms, total: 3min 52s
Wall time: 2min 8s


In [67]:
def get_embedding(word):
    try:
        embedding=model[word]
    except:
        embedding=np.zeros((300,))
    return embedding

In [68]:
#vocab size
len(model.wv.vocab)
#We have now represented each of 122248 words by a 100dim vector.

46335

In [69]:
X_train_embeddings=[np.mean(np.array(list(map(get_embedding,tok_sent))),axis=0) for tok_sent in X_train]
X_test_embeddings=[np.mean(np.array(list(map(get_embedding,tok_sent))),axis=0) for tok_sent in X_test]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Logistic Regression

In [70]:

"""
# multi_class
multi_class = ['multinomial']

# solver
solver = ['newton-cg', 'sag', 'saga', 'lbfgs']
 
# class_weight
class_weight = ['balanced', None]

# penalty
penalty = ['l2']

# Create the random grid
random_grid = {
               'multi_class': multi_class,
               'solver': solver,
               'class_weight': class_weight,
               'penalty': penalty}

"""

"\n# multi_class\nmulti_class = ['multinomial']\n\n# solver\nsolver = ['newton-cg', 'sag', 'saga', 'lbfgs']\n \n# class_weight\nclass_weight = ['balanced', None]\n\n# penalty\npenalty = ['l2']\n\n# Create the random grid\nrandom_grid = {\n               'multi_class': multi_class,\n               'solver': solver,\n               'class_weight': class_weight,\n               'penalty': penalty}\n\n"

In [71]:
"""

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
# First create the base model to tune
lrc = LogisticRegression(random_state=8)

# Definition of the random search
random_search = RandomizedSearchCV(estimator=lrc,
                                   param_distributions=random_grid,
                                   n_iter=50,
                                   scoring='accuracy',
                                   cv=3, 
                                   verbose=1, 
                                   random_state=8)

# Fit the random search model
random_search.fit(X_train_embeddings,y_train)

"""

"\n\nfrom sklearn.model_selection import RandomizedSearchCV\nfrom sklearn.model_selection import GridSearchCV\n# First create the base model to tune\nlrc = LogisticRegression(random_state=8)\n\n# Definition of the random search\nrandom_search = RandomizedSearchCV(estimator=lrc,\n                                   param_distributions=random_grid,\n                                   n_iter=50,\n                                   scoring='accuracy',\n                                   cv=3, \n                                   verbose=1, \n                                   random_state=8)\n\n# Fit the random search model\nrandom_search.fit(X_train_embeddings,y_train)\n\n"

In [72]:
"""
print("The best hyperparameters from Random Search are:")
print(random_search.best_params_)
print("")
print("The mean accuracy of a model with these hyperparameters is:")
print(random_search.best_score_)

"""


'\nprint("The best hyperparameters from Random Search are:")\nprint(random_search.best_params_)\nprint("")\nprint("The mean accuracy of a model with these hyperparameters is:")\nprint(random_search.best_score_)\n\n'

In [73]:
from sklearn.linear_model import LogisticRegression
# First create the base model to tune
logreg = LogisticRegression(n_jobs=-1,solver='newton-cg',penalty='l2',multi_class='multinomial',class_weight='balanced')

# Fit the random search model
logreg.fit(X_train_embeddings,y_train)

ValueError: ignored

In [ ]:
y_pred_test_log=logreg.predict(X_test_embeddings)
y_pred_train_log=logreg.predict(X_train_embeddings)

In [ ]:
d=accuracy_score(y_pred_test_log,y_test)*100
print ('Test Accuracy',d)

e=accuracy_score(y_pred_train_log,y_train)*100
print ('Train Accuracy',e)

In [ ]:
print(classification_report(y_test, y_pred_test_log))

SGD

In [ ]:
%%time
sgd_search = SGDClassifier(n_jobs=-1,alpha=0.00001)
sgd_search.fit(X_train_embeddings,y_train)

y_pred_test_sgd = sgd_search.predict(X_test_embeddings)
y_pred_train_sgd = sgd_search.predict(X_train_embeddings)


In [ ]:
f=accuracy_score(y_pred_test_sgd,y_test)*100
print ('Test Accuracy',f)

g=accuracy_score(y_pred_train_sgd,y_train)*100
print ('Train Accuracy',g)

In [ ]:
print(classification_report(y_test, y_pred_test_sgd))

In [ ]:
# title-category function
def title_to_category1(title):
    title=clean_text(title)
    print(title)

    title=clean_symbol(title)
    print(title)

    title=clean_shortforms(title)
    print(title)

    title=sent_tokenize(title)
    print(title)


    categories = {'b' : 'business', 
                  't' : 'science and technology', 
                  'e' : 'entertainment', 
                  'm' : 'health'}
    a=[np.mean(np.array(list(map(get_embedding,tok_sent))),axis=0) for tok_sent in title]
    pridicter1 = sgd_search.predict(a)
    print(pridicter1)
    return categories[max(pridicter1)]

In [ ]:
title_to_category1('Total of 6,666 abortions carried out under new legislation last year')